# Subgrounds Tutorials: SyntheticFields: RocketPool Exploration

In this example, we are using  SyntheticFields to create new fields that do not exist in the original data source. These new fields are calculated based on existing fields in the financialsDailySnapshots entity.

First, we create a FieldPath object to query the required fields from the financialsDailySnapshots entity, sorted by timestamp in descending order and limited to the last 30 entries.

Next, we create a synthetic field datetime in the FinancialsDailySnapshot entity to convert the timestamp field into a human-readable format.

Then, we define three more synthetic fields. 
- `revDiffRevenueUSD30` calculates the difference in revenue between the dailyProtocolSideRevenueUSD and dailySupplySideRevenueUSD fields over the past 30 days. 
- `avgDailyRevenueUSD30` calculates the average of the dailyProtocolSideRevenueUSD and dailySupplySideRevenueUSD fields over the past 30 days.-
- `revenueMargin30` calculates the percentage change in revenue between the dailyProtocolSideRevenueUSD and dailySupplySideRevenueUSD fields over the past 30 days.

Finally, we query the required fields, including the newly created synthetic fields, and flatten the data into a single DataFrame for analysis.


In [1]:
from datetime import datetime
from subgrounds.subgraph import SyntheticField
from subgrounds import Subgrounds
import pandas as pd

# Initialize Subgrounds
sg = Subgrounds()

# Load a subgraph using its API URL
financials_daily_snapshots = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/messari/rocket-pool-ethereum')

# Create a FieldPath object for the required fields in the financialsDailySnapshots entity.
# Specify options to sort the data by timestamp in descending order and limit the number of results to 30.
financials_daily_snapshot_30days = financials_daily_snapshots.Query.financialsDailySnapshots(
    orderBy=financials_daily_snapshots.FinancialsDailySnapshot.timestamp, 
    orderDirection='desc', 
    first=30
)

# Create a SyntheticField on the Swap entity called `timestamp`, which will format the timestamp field into something more human readable
financials_daily_snapshots.FinancialsDailySnapshot.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.FLOAT,
  financials_daily_snapshots.FinancialsDailySnapshot.timestamp
)

# Define a synthetic field to calculate the daily revenue difference for the past 30 days
financials_daily_snapshots.FinancialsDailySnapshot.revDiffRevenueUSD30 = SyntheticField(
    lambda x, y: abs(x - y),
    SyntheticField.FLOAT,
    [financials_daily_snapshots.FinancialsDailySnapshot.dailyProtocolSideRevenueUSD,
    financials_daily_snapshots.FinancialsDailySnapshot.dailySupplySideRevenueUSD],
)

# the average daily revenue
financials_daily_snapshots.FinancialsDailySnapshot.avgDailyRevenueUSD30 = SyntheticField(
    lambda x, y: (x + y) / 2,
    SyntheticField.FLOAT,
    [financials_daily_snapshots.FinancialsDailySnapshot.dailyProtocolSideRevenueUSD,
    financials_daily_snapshots.FinancialsDailySnapshot.dailySupplySideRevenueUSD],
)

# Create a SyntheticField for the percentage change in daily revenue over the past 30 days
financials_daily_snapshots.FinancialsDailySnapshot.revenueMargin30  = SyntheticField(
    lambda x, y: x / y,
    SyntheticField.FLOAT,
    [financials_daily_snapshots.FinancialsDailySnapshot.dailyProtocolSideRevenueUSD,
    financials_daily_snapshots.FinancialsDailySnapshot.dailySupplySideRevenueUSD],
)


# Query data flattened to a single DataFrame
sg.query_df([
    financials_daily_snapshot_30days.timestamp,
    financials_daily_snapshot_30days.datetime,
    financials_daily_snapshot_30days.blockNumber,
    financials_daily_snapshot_30days.cumulativeProtocolSideRevenueUSD,
    financials_daily_snapshot_30days.cumulativeProtocolSideRevenueUSD,
    financials_daily_snapshot_30days.cumulativeSupplySideRevenueUSD,
    financials_daily_snapshot_30days.cumulativeTotalRevenueUSD,
    financials_daily_snapshot_30days.dailyProtocolSideRevenueUSD,
    financials_daily_snapshot_30days.dailySupplySideRevenueUSD,
    financials_daily_snapshot_30days.dailyTotalRevenueUSD,
    financials_daily_snapshot_30days.revDiffRevenueUSD30,
    financials_daily_snapshot_30days.avgDailyRevenueUSD30,
    financials_daily_snapshot_30days.revenueMargin30,

])

,financialsDailySnapshots_timestamp,financialsDailySnapshots_datetime,financialsDailySnapshots_blockNumber,financialsDailySnapshots_cumulativeProtocolSideRevenueUSD,financialsDailySnapshots_cumulativeSupplySideRevenueUSD,financialsDailySnapshots_cumulativeTotalRevenueUSD,financialsDailySnapshots_dailyProtocolSideRevenueUSD,financialsDailySnapshots_dailySupplySideRevenueUSD,financialsDailySnapshots_dailyTotalRevenueUSD,financialsDailySnapshots_revDiffRevenueUSD30,financialsDailySnapshots_avgDailyRevenueUSD30,financialsDailySnapshots_revenueMargin30
0,1678223279,2023-03-07 15:07:59,16778991,1.537598e+06,9.205565e+06,1.074316e+07,9401.209628,54871.696408,64272.906036,45470.486780,32136.453018,0.171331
1,1678083023,2023-03-06 00:10:23,16767448,1.528196e+06,9.150693e+06,1.067889e+07,5252.416150,30656.585248,35909.001398,25404.169098,17954.500699,0.171331
2,1678013171,2023-03-05 04:46:11,16761693,1.522944e+06,9.120037e+06,1.064298e+07,9403.072566,54882.569751,64285.642317,45479.497184,32142.821158,0.171331
3,1677873035,2023-03-03 13:50:35,16750163,1.513541e+06,9.065154e+06,1.057870e+07,10590.856681,61815.265850,72406.122531,51224.409170,36203.061266,0.171331
4,1677732803,2023-03-01 22:53:23,16738636,1.502950e+06,9.003339e+06,1.050629e+07,4959.309763,28945.821920,33905.131683,23986.512156,16952.565841,0.171331
5,1677662879,2023-03-01 03:27:59,16732881,1.497991e+06,8.974393e+06,1.047238e+07,10129.221629,59120.857428,69250.079057,48991.635799,34625.039529,0.171331
6,1677523139,2023-02-27 12:38:59,16721366,1.487861e+06,8.915272e+06,1.040313e+07,4698.096833,27421.209963,32119.306796,22723.113130,16059.653398,0.171331
7,1677453299,2023-02-26 17:14:59,16715608,1.483163e+06,8.887851e+06,1.037101e+07,9636.410818,56244.486589,65880.897407,46608.075772,32940.448704,0.171331
8,1677313487,2023-02-25 02:24:47,16704101,1.473527e+06,8.831607e+06,1.030513e+07,5255.910699,30676.981752,35932.892450,25421.071053,17966.446225,0.171331
9,1677243323,2023-02-24 06:55:23,16698326,1.468271e+06,8.800930e+06,1.026920e+07,4780.268922,27900.820790,32681.089712,23120.551868,16340.544856,0.171331
